# Voice-to-Report System for First Responders

This notebook captures spoken incident reports and generates a structured report.

## Setup
- Installed: `speechrecognition`, `PyAudio`
- Ensure a microphone is connected.
- Run cells in sequence.

In [2]:
import sys
print(sys.executable)

/usr/local/bin/python3


In [13]:
import speech_recognition as sr
print(sr.__version__)

3.14.3


In [4]:
from datetime import datetime
import re
from IPython.display import display, Markdown

In [5]:
recognizer = sr.Recognizer()

def capture_voice_input():
    """Capture voice input from microphone and return transcribed text."""
    with sr.Microphone() as source:
        print("Listening for incident report... (Speak clearly, say 'stop' to finish)")
        recognizer.adjust_for_ambient_noise(source, duration=2)  # Adjust for noise, longer duration
        audio_data = []
        
        while True:
            try:
                print("Recording... Say your report or 'stop' to finish.")
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=15)  # Increased phrase limit
                print("Processing audio...")
                text = recognizer.recognize_google(audio)
                print(f"Transcribed: '{text}'")  # Debug: Show what was transcribed
                # Check for variations of 'stop' (case-insensitive, partial match)
                if 'stop' in text.lower():
                    print("Detected 'stop'. Ending recording.")
                    break
                audio_data.append(text)
            except sr.WaitTimeoutError:
                print("No speech detected for 5 seconds. Still listening...")
            except sr.UnknownValueError:
                print("Could not understand audio, please repeat.")
            except sr.RequestError as e:
                print(f"Speech recognition error: {e}")
                return None
            except Exception as e:
                print(f"Unexpected error: {e}")
                return None
        return " ".join(audio_data) if audio_data else None

In [6]:
transcription = capture_voice_input()
print("Final transcribed text:", transcription)

Listening for incident report... (Speak clearly, say 'stop' to finish)
Recording... Say your report or 'stop' to finish.
Processing audio...
Transcribed: 'hello'
Recording... Say your report or 'stop' to finish.
Processing audio...
Transcribed: 'incident on Main Street stop'
Detected 'stop'. Ending recording.
Final transcribed text: hello


In [7]:
def parse_transcription(text):
    """Parse transcribed text into structured report fields."""
    report = {
        "incident_type": "Not specified",
        "location": "Not specified",
        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "description": text,
        "actions_taken": "Not specified"
    }
    text_lower = text.lower()
    if "fire" in text_lower:
        report["incident_type"] = "Fire"
    elif "medical" in text_lower or "injury" in text_lower:
        report["incident_type"] = "Medical Emergency"
    elif "accident" in text_lower or "crash" in text_lower:
        report["incident_type"] = "Traffic Accident"
    location_match = re.search(r"(?:at|in|near)\s+([a-zA-Z0-9\s,]+)", text_lower)
    if location_match:
        report["location"] = location_match.group(1).capitalize()
    return report

In [8]:
if transcription:
    report_data = parse_transcription(transcription)
    print("Parsed report:", report_data)
else:
    print("No transcription available to parse.")

Parsed report: {'incident_type': 'Not specified', 'location': 'Not specified', 'time': '2025-06-20 20:12:09', 'description': 'hello', 'actions_taken': 'Not specified'}


In [9]:
def generate_report(report_data):
    """Generate a markdown report from structured data."""
    markdown = f"""# Incident Report
**Date and Time:** {report_data['time']}
**Incident Type:** {report_data['incident_type']}
**Location:** {report_data['location']}
**Description:** {report_data['description']}
**Actions Taken:** {report_data['actions_taken']}
"""
    return markdown

In [10]:
def save_report(markdown_content, filename="incident_report.md"):
    """Save the report to a markdown file."""
    with open(filename, "w") as file:
        file.write(markdown_content)
    print(f"Report saved as {filename}")

In [17]:
import os
print(os.getcwd())

/Users/amyz/Desktop/calhacks


## Example Usage
1. Run the final cell.
2. Speak clearly, e.g., "Fire at 456 Oak Avenue, smoke detected, stop."
3. Check the displayed report and saved `incident_report.md`.

## Example Output
For input: "Medical emergency at 123 Main Street, patient unconscious, stop."
```markdown
# Incident Report
**Date and Time:** 2025-06-20 16:57:23
**Incident Type:** Medical Emergency
**Location:** 123 Main Street
**Description:** Medical emergency at 123 Main Street, patient unconscious
**Actions Taken:** Not specified

In [11]:
transcription = capture_voice_input()
if not transcription:
    print("No valid transcription captured. Please try again.")
else:
    report_data = parse_transcription(transcription)
    report_content = generate_report(report_data)
    display(Markdown(report_content))
    save_report(report_content)

Listening for incident report... (Speak clearly, say 'stop' to finish)
Recording... Say your report or 'stop' to finish.
Processing audio...
Transcribed: 'turn on Main Street'
Recording... Say your report or 'stop' to finish.
Processing audio...
Transcribed: 'calling for backup'
Recording... Say your report or 'stop' to finish.
Processing audio...
Could not understand audio, please repeat.
Recording... Say your report or 'stop' to finish.
Processing audio...
Transcribed: 'stop'
Detected 'stop'. Ending recording.


# Incident Report
**Date and Time:** 2025-06-20 20:12:38
**Incident Type:** Not specified
**Location:** Street calling for backup
**Description:** turn on Main Street calling for backup
**Actions Taken:** Not specified


Report saved as incident_report.md


In [14]:
p = pyaudio.PyAudio()

# List available audio devices
print("Available audio devices:")
for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print(f"Device {i}: {dev['name']}")

# Terminate PyAudio
p.terminate()

NameError: name 'pyaudio' is not defined